In [ ]:
import os 
import pandas as pd
import io
import openai
from transformers.models.imagegpt.modeling_imagegpt import IMAGEGPT_INPUTS_DOCSTRING
from transformers import GPT2Tokenizer
import pandas as pd 
from pandas.io import json
from numpy import nan
import time
import csv
import shutil
import datetime
import pytz
import re
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
import time
import random
import itertools
import random

In [ ]:
def get_combinations(prompt_paths, model_engines, datasets, folds_LLL, folds_HPRD50, folds_IEPA):
    combinations = []

    # Create dictionary for dataset-specific folds
    dataset_folds = {
        'LLL': folds_LLL,
        'HPRD50': folds_HPRD50,
        'IEPA': folds_IEPA
    }

    # Exclude 'BASE' prompts for 'LLL'
    prompt_paths_LLL = [path for path in prompt_paths if "BASE" not in path]

    # Iterate over each dataset to get the dataset-specific folds and prompts
    for dataset in datasets:
        current_prompts = prompt_paths_LLL if dataset == "LLL" else prompt_paths
        current_folds = dataset_folds[dataset]
        
        # Use itertools.product to create combinations for the current dataset
        for combination in itertools.product(current_prompts, model_engines, [dataset], current_folds):
            combinations.append(combination)
    # Shuffle the list of combinations
    random.shuffle(combinations)
    
    return combinations

In [ ]:
def output_path(Run_no, dataset, temperature, prompt_type, model_engine):
    base = "Output/" + model_engine + "_Final_Prompts_" + str(temperature) + "/Prompt" + str(prompt_type) +"/"+dataset + "/"
    extension_path = dataset + "_T" + str(temperature) + "_" + prompt_type + "_Run" + str(Run_no)+'/'
    Implementation_base_path_output = os.path.join(base, extension_path)
    os.makedirs(Implementation_base_path_output, exist_ok=True)
    
    return Implementation_base_path_output

In [ ]:
def normalized_Entity(str):
  str = str.replace('\t', '')
  str = str.replace('"', '')
  str = str.replace('(', '')

  str = str.replace(')', '')
  str = str.replace(' ', '')
  str = re.sub(r'\s', '', str)
  str = str.replace('~', '')
  str = str.replace('-', '')
  str = str.replace(',', '')
  str = str.lower()
  str = str.replace('operon', '')
  str = str.replace('regulon', '')
  str = str.replace('genes', '')
  str = str.replace('gene', '')
  str = str.replace('mutant', '')
  str = str.replace('phosphatase', '')
  str = str.replace('complex', '')
  str = str.strip()

  return str


In [ ]:
def split_string_by_slash(input_string):
    parts = input_string.split("/")
    return parts

def split_string_by_comma(input_string):
    parts = input_string.split(",")
    return parts

def check_word_in_predicted_entity(predicted, original):
    pattern = r'\b{}\b'.format(re.escape(original))
    return bool(re.search(pattern, predicted))

def sort_entities(row):
        return tuple(sorted([row['E1'], row['E2']]))


In [ ]:
def evaluation(data_path, run, output_path, interaction_processed):#, Norm_Predicted_Protein_Sciminer):

    #skipped_lines_total = 0
    for f in range(10):
        fold = f + 1
        print(run[f])
        print(data_path)
        print(os.path.join(data_path, run[f]))

        with open(os.path.join(data_path, run[f]), 'r') as file:
            # Filter out blank lines and store them in a list
            lines = [line for line in file if line.strip()]
            # Now total_lines should be the count of the filtered lines
            total_lines = len(lines)    
        

        # Read the first line of the CSV
        with open(os.path.join(data_path, run[f]), 'r') as file:
            first_line = file.readline().strip()

        # Check if the first line contains "Output:"
        if "Output:" in first_line:
            # Read the file skipping the first row and using comma as separator
            df_predicted = pd.read_csv(os.path.join(data_path, run[f]), sep=',', skiprows=1, on_bad_lines='skip')
        else:
            # Directly read the file using comma as separator
            df_predicted = pd.read_csv(os.path.join(data_path, run[f]), sep=',', on_bad_lines='skip')

        
        # Subtracting 1 for the header row
        skipped_lines = total_lines - df_predicted.shape[0] - 1 if total_lines > 0 else 0

        print(f"Number of lines skipped: {skipped_lines}")
        #skipped_lines_total = skipped_lines + skipped_lines_total
        
        # Check if the 'status' column of the last row contains 'DONE'
        if 'done' in map(str.lower, map(str, df_predicted.iloc[-1].values)):
            df_predicted = df_predicted.iloc[:-1]
            
        print(df_predicted)
        
        df_predicted = df_predicted.dropna()
        #df_predicted.columns = ['Sentence_ID', 'E1', 'E2', 'Interaction']
        df_predicted.columns = ['Sentence_ID', 'E1', 'E2', 'Interaction']
        df_predicted = df_predicted[~df_predicted['Interaction'].str.contains("not ")]   ###########

        df_predicted['Sorted_Entities'] = df_predicted.apply(sort_entities, axis=1)

        df_combined_interactions = df_predicted.groupby(['Sentence_ID', 'Sorted_Entities'])['Interaction'].apply(', '.join).reset_index()

        df_combined_interactions_deduplicated = df_combined_interactions.drop_duplicates(subset=['Sentence_ID', 'Sorted_Entities'], keep='first')
        df_combined_interactions_deduplicated = df_combined_interactions_deduplicated[~df_combined_interactions_deduplicated.apply(lambda row: row.astype(str).str.contains("Done").any(), axis=1)]
        df_combined_interactions_deduplicated = df_combined_interactions_deduplicated.reset_index(drop=True)

        df_combined_interactions_deduplicated[['E1', 'E2']] = pd.DataFrame(df_combined_interactions_deduplicated['Sorted_Entities'].tolist(), index=df_combined_interactions_deduplicated.index)
        df_predicted_processed = df_combined_interactions_deduplicated.drop(columns=['Sorted_Entities'])
        # Resetting the index of the DataFrame
        df_predicted_processed.reset_index(drop=True, inplace=True)  ########
        No_of_predicted_interaction_pairs = len(df_predicted_processed.index)
        print(No_of_predicted_interaction_pairs)
        TP = 0
        Partial_TP = 0
        for i in range(len(df_predicted_processed.index)):
            match_found = False
            partial_match_found = False
            for j in range(len(interaction_processed.index)):
                Predicted_Sentence_ID = df_predicted_processed.iloc[i]["Sentence_ID"]
                Predicted_Sentence_ID = re.sub(r'\s', '', Predicted_Sentence_ID)
                
                #Norm_Predicted_SID = Norm_Predicted_Protein_Sciminer[Norm_Predicted_Protein_Sciminer["Sentence_ID"] == Predicted_Sentence_ID]
                #Norm_Original_SID = Norm_Original_Protein_Sciminer[Norm_Original_Protein_Sciminer["Sentence_ID"] == Predicted_Sentence_ID]

                if interaction_processed.iloc[j]["Fold"] == fold and Predicted_Sentence_ID == interaction_processed.iloc[j]["sen_id"]:
                    str1_raw = df_predicted_processed.iloc[i]["E1"]
                    str2_raw = df_predicted_processed.iloc[i]["E2"]

                    str3_raw = interaction_processed.iloc[j]["E1"]
                    str4_raw = interaction_processed.iloc[j]["E2"]
                    

                    str1 = normalized_Entity(str1_raw)
                    str2 = normalized_Entity(str2_raw)
                    str3 = normalized_Entity(str3_raw)
                    str4 = normalized_Entity(str4_raw)

                    result1 = split_string_by_slash(str1_raw)
                    result2 = split_string_by_slash(str2_raw)
                    
                    result1_comma = split_string_by_comma(str1_raw)
                    result2_comma = split_string_by_comma(str2_raw)

                    if len(result1) > 1:
                        for part in result1:
                            normalized_part = normalized_Entity(part)
                            if (normalized_part == str3 and str2 == str4) or (normalized_part == str4 and str2 == str3):
                                match_found = True
                                break
                                                             
                    elif len(result2) > 1:
                        for part in result2:
                            normalized_part = normalized_Entity(part)
                            if (str1 == str3 and normalized_part == str4) or (str1 == str4 and normalized_part == str3):
                                match_found = True
                                break
                            
                                
                    if len(result1_comma) > 1:
                        for part in result1_comma:
                            normalized_part = normalized_Entity(part)
                            if (normalized_part == str3 and str2 == str4) or (normalized_part == str4 and str2 == str3):
                                print(str1, str2, str3, str4)
                                match_found = True
                                break
                                                  
                    elif len(result2_comma) > 1:
                        for part in result2_comma:
                            normalized_part = normalized_Entity(part)
                            if (str1 == str3 and normalized_part == str4) or (str1 == str4 and normalized_part == str3):
                                print(str1, str2, str3, str4)
                                match_found = True
                                break
                                
                    elif (str1 == str3 and str2 == str4) or (str1 == str4 and str2 == str3):
                        match_found = True
                        break
                    elif check_word_in_predicted_entity(str1_raw.lower(), str3_raw.lower()) and check_word_in_predicted_entity(str2_raw.lower(), str4_raw.lower()):
                        partial_match_found = True
                        break
                    elif check_word_in_predicted_entity(str1_raw.lower(), str3_raw.lower()) and str2 == str4:
                        partial_match_found = True
                        break
                    elif str1 == str3 and check_word_in_predicted_entity(str2_raw.lower(), str4_raw.lower()):
                        partial_match_found = True
                        break
                    elif check_word_in_predicted_entity(str1_raw.lower(), str4_raw.lower()) and check_word_in_predicted_entity(str2_raw.lower(), str3_raw.lower()):
                        partial_match_found = True
                        break
                    elif check_word_in_predicted_entity(str1_raw.lower(), str4_raw.lower()) and str2 == str3:
                        partial_match_found = True
                        break
                    elif str1 == str4 and check_word_in_predicted_entity(str2_raw.lower(), str3_raw.lower()):
                        partial_match_found = True
                        break
                    
            if match_found:
                TP += 1
            if partial_match_found:
                TP += 1
               

        interaction_processed_unique = interaction_processed.drop_duplicates()
        No_of_original_P_interaction_pairs = (interaction_processed_unique['Fold'] == fold).sum()
        
        
            
        if TP > No_of_original_P_interaction_pairs:
            TP = No_of_original_P_interaction_pairs
        print("predicted_pairs, original_P_interaction_pairs, TP:",No_of_predicted_interaction_pairs, No_of_original_P_interaction_pairs, TP)
          
            
        Partial_Precision = (TP)/No_of_predicted_interaction_pairs
        Partial_Recall = (TP)/ No_of_original_P_interaction_pairs  
        Partial_F1 = 0 

        if Partial_Precision + Partial_Recall > 0:
          Partial_F1 = 2 * (Partial_Precision * Partial_Recall)/(Partial_Precision + Partial_Recall)
                                           
        with open(output_path, "a") as f:
          print (data_path, fold,',', "{:.4f}".format(Partial_Precision), ',', "{:.4f}".format(Partial_Recall), ',',"{:.4f}".format(Partial_F1), ',', skipped_lines, file=f)


In [ ]:
def process_interactions(dataset, base_path_output, model_engine, temperature, prompt_type, run_count=10):

    runs = {i: [f'{i}_fold{j}.txt' for j in range(1, 11)] for i in range(1, 11)}

    output_paths = {
        i: f'{base_path_output}/Evaluation_{dataset}_T{temperature}_{prompt_type}_Run{i}.csv'
        for i in range(1, run_count + 1)
    }


    implementation_base_path_output_runs = {}

    implementation_base_path_output_runs = {
        i: output_path(i, dataset, temperature, prompt_type, model_engine) for i in range(1, 11)
    }

    if dataset == "LLL":
        interaction = pd.read_csv("Datasets/LLL/LLL_ALL_P_interaction_10fold.csv", sep=',')     ### Careful ::::: Run No
        interaction = interaction.rename(columns={'Sentence_ID': 'sen_id'})
        # Step 1: Sort 'E1' and 'E2' columns within each row to create a new column 'Sorted_Entities'
        interaction['Sorted_Entities'] = interaction.apply(lambda row: tuple(sorted([row['E1'], row['E2']])), axis=1)

        # Step 2: Drop duplicates based on 'Sentence_ID' and 'Sorted_Entities'
        interaction_processed = interaction.drop_duplicates(subset=['sen_id', 'Sorted_Entities'], keep='first')

        # Step 3: Drop the temporary 'Sorted_Entities' column
        interaction_processed = interaction_processed.drop(columns=['Sorted_Entities'])

        interaction_processed = interaction_processed.reset_index(drop=True)

        print(len(interaction))
        print(len(interaction_processed))


    elif dataset == "HPRD50":
        #base_path = Implementation_base_path_output
        interaction = pd.read_csv("Datasets/HPRD50/interaction_processed_10fold.csv", sep=',')     ### Careful ::::: Run No
        interaction = interaction[interaction["isValid"] == True]
        interaction = interaction.rename(columns={'Sentence_ID': 'sen_id'})

        # Step 1: Sort 'E1' and 'E2' columns within each row to create a new column 'Sorted_Entities'
        interaction['Sorted_Entities'] = interaction.apply(lambda row: tuple(sorted([row['E1'], row['E2']])), axis=1)

        # Step 2: Drop duplicates based on 'Sentence_ID' and 'Sorted_Entities'
        interaction_processed = interaction.drop_duplicates(subset=['sen_id', 'Sorted_Entities'], keep='first')

        # Step 3: Drop the temporary 'Sorted_Entities' column
        interaction_processed = interaction_processed.drop(columns=['Sorted_Entities'])

        interaction_processed = interaction_processed.reset_index(drop=True)

        print("\n\n\ninteraction:",len(interaction))
        print("\n\n\ninteraction processed:",len(interaction_processed))




    elif dataset == "IEPA":
        df_sentence_dictionary = pd.read_csv("Datasets/IEPA/sentence_dictionary_10fold.csv", sep=',')  
        interaction = pd.read_csv("Datasets/IEPA/IEPA_all_PN_interaction_final.csv", sep=',')     ### Careful ::::: Run No
        interaction_processed = interaction[interaction["isValid"] == True]
        df_sentence_dictionary = df_sentence_dictionary.rename(columns={'Sentence_ID': 'sen_id', 'Fold_no': 'Fold'})
        # drop the 'C' column
        df_sentence_dictionary = df_sentence_dictionary.drop('Sentence', axis=1)
        interaction_processed = pd.merge(interaction_processed, df_sentence_dictionary, on='sen_id', how='inner')

        # Step 1: Sort 'E1' and 'E2' columns within each row to create a new column 'Sorted_Entities'
        interaction_processed['Sorted_Entities'] = interaction_processed.apply(lambda row: tuple(sorted([row['E1'], row['E2']])), axis=1)
        # Step 2: Drop duplicates based on 'Sentence_ID' and 'Sorted_Entities'
        interaction_processed = interaction_processed.drop_duplicates(subset=['sen_id', 'Sorted_Entities'], keep='first')
        # Step 3: Drop the temporary 'Sorted_Entities' column
        interaction_processed = interaction_processed.drop(columns=['Sorted_Entities'])
        interaction_processed = interaction_processed.reset_index(drop=True)
        print("\n\n\ninteraction:",len(interaction))
        print("\n\n\ninteraction processed:",len(interaction_processed))    
    else:
        interaction_processed = 'Null'
        print("Sorry the true PPI for this dataset is not available now.")


    for i in range(1, 11):
        print(implementation_base_path_output_runs)
        print(runs)
        print(output_paths)
        evaluation(implementation_base_path_output_runs[i], runs[i], output_paths[i], interaction_processed)

In [ ]:
def summarize_evaluation(folder_path, temperature):
    # Create an empty DataFrame to store the results
    results_df = pd.DataFrame(columns=["filename", "average precision", "average recall", "average f1 score", "total bad lines"])

    # Loop through the CSV files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            # Read the CSV file
            df = pd.read_csv(os.path.join(folder_path, filename), header=None, names=["filename", "precision", "recall", "f1 score", "total bad lines"])

            # Calculate averages and total bad lines
            avg_precision = df["precision"].mean() * 100
            avg_recall = df["recall"].mean() * 100
            avg_f1_score = df["f1 score"].mean() * 100
            total_bad_lines = df["total bad lines"].sum()

            # Append results to the DataFrame
            results_df = results_df.append({
                "filename": filename,
                "average precision": avg_precision,
                "average recall": avg_recall,
                "average f1 score": avg_f1_score,
                "total bad lines": total_bad_lines
            }, ignore_index=True)

    # Extract "T_value" from filenames
    results_df["T_value"] = results_df["filename"].str.extract("T(\d+\.\d+)")

    # Select relevant columns for a shortened DataFrame
    shortened_df = results_df[["T_value", "average precision", "average recall", "average f1 score", "total bad lines"]].sort_values(by="T_value")

    # Calculate overall averages
    avg_precision = round(shortened_df['average precision'].mean(), 2)
    avg_recall = round(shortened_df['average recall'].mean(), 2)
    avg_f1 = round(shortened_df['average f1 score'].mean(), 2)
    total_bad_lines = shortened_df['total bad lines'].mean()

    # Append a row with the overall averages
    new_row = {'T_value': temperature, 'average precision': avg_precision, 'average recall': avg_recall, 'average f1 score': avg_f1, "total bad lines": total_bad_lines}
    shortened_df = shortened_df.append(new_row, ignore_index=True)

    # Save the summarized data to a CSV file
    filepath = os.path.join(folder_path, 'Average_Score.csv')
    shortened_df.to_csv(filepath, index=False, float_format='%.2f')

    # Return the summarized DataFrame
    return shortened_df

In [ ]:
if __name__ == '__main__':
    prompt_types = ['BASE', 'WD', 'WND']
    model_engines = ["gpt-3.5-turbo-0613", "gpt-4-0613"]
    datasets = ["LLL", "HPRD50", "IEPA"]
    temperature = 0.0

    for dataset in datasets:
        for model_engine in model_engines:
            for prompt_type in prompt_types:
                
                print(f"Processing: Dataset={dataset}, Model={model_engine}, Prompt={prompt_type}, Temperature={temperature}")
                base_path_output = f'Output/{model_engine}_Final_Prompts_{temperature}/Prompt{prompt_type}/{dataset}/'

                # Process interactions
                interaction_processed = process_interactions(dataset, base_path_output, model_engine, temperature, prompt_type)
                print("Processed interactions:", interaction_processed)
                # Summarize evaluation
                summarized_data = summarize_evaluation(base_path_output, temperature)
                print("Summarized data:\n", summarized_data)
